# RAG 기법
RAG(Retrieval-Augmented Generation) 기법은 기존의 대규모 언어 모델(LLM)을 확장하여, 주어진 컨텍스트나 질문에 대해 더욱 정확하고 풍부한 정보를 제공하는 방법입니다. 
모델이 학습 데이터에 포함되지 않은 외부 데이터를 실시간으로 검색(retrieval)하고, 이를 바탕으로 답변을 생성(generation)하는 과정을 포함합니다. 특히 환각(생성된 내용이 사실이 아닌 것으로 오인되는 현상)을 방지하고, 모델이 최신 정보를 반영하거나 더 넓은 지식을 활용할 수 있게 합니다.

1. RAG 모델의 기본 구조
    - 검색 단계(Retrieval Phase)
     - 사용자의 질문이나 컨텍스트를 입력으로 받아서, 이와 관련된 외부 데이터를 검색하는 단계입니다. 이 때 검색 엔진이나 데이터베이스 등 다양한 소스에서 필요한 정보를 찾아냅니다. 검색된 데이터는 질문에 대한 답변을 생성하는데 적합하고 상세한 정보를 포함하는 것을 목표로 합니다.

    - 생성 단계(Generation Phase)
      - 검색된 데이터를 기반으로 LLM 모델이 사용자의 질문에 답변을 생성하는 단계입니다. 이 단계에서 모델은 검색된 정보와 기존의 지식을 결합하여, 주어진 질문에 대한 답변을 생성합니다.
2. RAG 모델의 장점
    - 풍부한 정보 제공
      - RAG 모델은 검색을 통해 얻은 외부 데이터를 활용하여, 보다 구체적이고 풍부한 정보를 제공할 수 있습니다.
    - 실시간 정보 반영
      - 최신 데이터를 검색하여 반영함으로써, 모델이 실시간으로 변화하는 정보에 대응할 수 있습니다.
    - 환각 방지
      - 검색을 통해 실제 데이터에 기반한 답변을 생성함으로써, 환각 현상이 발생할 위험을 줄이고 정확도를 높일 수 있습니다.

## RAG 개요
RAG(Retrieval-Augmented Generation) 파이프라인은 기존의 언어 모델에 검색 기능을 추가하여, 주어진 질문이나 문제에 대해 더 정확하고 풍부한 정보를 기반으로 답변을 생성할 수 있게 해줍니다. 
이 파이프라인은 크게 `데이터 로드`, `텍스트 분할`, `인덱싱`, `검색`, `생성`의 다섯 단계로 구성됩니다\

1. 데이터 로드(Load Data)
   RAG에 사용할 데이터를 불러오는 단계입니다. 
   외부 데이터 소스에서 정보를 수집하고, 필요한 형식으로 변환하여 시스템에 로드합니다. 
   
   예를 들면 공개 데이터셋, 웹 크롤링을 통해 얻은 데이터, 또는 사전에 정리된 자료일 수 있습니다. 가져온 데이터는 검색에 사용될 지식이나 정보를 담고 있어야 합니다.

In [70]:
from IPython.display import Image
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Documents
docs = loader.load()

print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])


1
13307
체적으로 어기고 있다면 규범 준수를 위해 좀 더 빠르게 강력한 수단을 이용해야 합니다. 특히 정책 문서에 명시된 원칙을 지키지 않는 것은 대부분의 경우 다른 사용자에게 받아들여지지 않습니다 (다른 분들에게 예외 상황임을 설득할 수 있다면 가능하기는 하지만요). 이는 당신을 포함해서 편집자 개개인이 정책과 지침을 직접 집행 및 적용한다는 것을 의미합니다.
특정 사용자가 명백히 정책에 반하는 행동을 하거나 정책과 상충되는 방식으로 지침을 어기는 경우, 특히 의도적이고 지속적으로 그런 행위를 하는 경우 해당 사용자는 관리자의 제재 조치로 일시적, 혹은 영구적으로 편집이 차단될 수 있습니다. 영어판을 비롯한 타 언어판에서는 일반적인 분쟁 해결 절차로 끝낼 수 없는 사안은 중재위원회가 개입하기도 합니다.
문서 내용
정책과 지침의 문서 내용은 처음 읽는 사용자라도 원칙과 규범을 잘 이해할 수 있도록 다음 원칙을 지켜야 합니다.
명확하게 작성하세요. 소수만 알아듣거나 준법률적인 단어, 혹은 지나치게 단순한 표현은 피해야 합니다. 명확하고, 직접적이고, 모호하지 않고, 구체적으로 작성하세요. 지나치게 상투적인 표현이나 일반론은 피하세요. 지침, 도움말 문서 및 기타 정보문 문서에서도 "해야 합니다" 혹은 "하지 말아야 합니다" 같이 직접적인 표현을 굳이 꺼릴 필요는 없습니다.
가능한 간결하게, 너무 단순하지는 않게. 정책이 중언부언하면 오해를 부릅니다. 불필요한 말은 생략하세요. 직접적이고 간결한 설명이 마구잡이식 예시 나열보다 더 이해하기 쉽습니다. 각주나 관련 문서 링크를 이용하여 더 상세히 설명할 수도 있습니다.
규칙을 만든 의도를 강조하세요. 사용자들이 상식대로 행동하리라 기대하세요. 정책의 의도가 명료하다면, 추가 설명은 필요 없죠. 즉 규칙을 '어떻게' 지키는지와 더불어 '왜' 지켜야 하는지 확실하게 밝혀야 합니다.
범위는 분명히, 중복은 피하기. 되도록 앞부분에서 정책 및 지침의 목적과 범위를 분명하게 밝혀야 합니다. 독자 대부분은 도입부 초반만 

2. 텍스트 분할
   불러온 데이터를 작은 크기의 단위(chunk)로 분할하는 과정입니다. 
   `자연어 처리(NLP) 기술을 활용하여 큰 문서를 처리가 쉽도록 문단, 문장 또는 구 단위로 나누는 작업입니다. `
   검색 효율성을 높이기 위한 중요한 과정입니다.

   다음 코드는 `RecursiveCharacterTextSplitter`라는 텍스트 분할 도구를 사용하고 있습니다. (이 도구에 대해서는 Text Splitter 챕터에서 상세하게 다룰 예정입니다. ) 간략하게 설명하면 12552 개의 문자로 이루어진 긴 문장을 최대 1000글자 단위로 분할하는 것입니다. 200글자는 각 분할마다 겹치게 하여 문맥이 잘려나가지 않고 유지되게 합니다. 실행 결과를 보면 18개 조각으로 나눠지게 됩니다.

   `LLM 모델이나 API의 입력 크기에 대한 제한`이 있기 때문에, 제한에 걸리지 않도록 적정한 크기로 텍스트의 길이를 줄일 필요가 있습니다. 그리고, 프롬프트가 지나치게 길어질 경우 중요한 정보가 상대적으로 희석되는 문제가 있을 수도 있습니다. 
   따라서, 적정한 크기로 텍스트를 분할하는 과정이 필요합니다.

In [4]:
# Text Split (Documents -> small chunks: Documents)
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[10])


16
page_content='정책과 지침 관련 총의는 추세가 뚜렷해야 하지만, 만장일치일 필요는 없습니다.
제안 작성자 이외에도 공동체 전체에 노출되어 있어야 합니다.
제안 문서의 영향력을 고려하세요.
토론 중 중대한 문제가 제기되었는가?
다른 정책 또는 지침과 모순되지는 않는가?
새로 제정된 다른 정책 또는 지침과 병합될 수도 있는가?
이미 존재하는 정책 및 지침과 중복되지는 않는가?
제안 처리는 단순 득표수로 결정되지 않습니다. 투표는 토론을 대체하지 않으며, 득표수가 총의와 합치하는 것도 아닙니다.
총의가 불분명하고, 이 상황이 개선될 것 같지 않다면 이는 거부됩니다.
채택 토론은 세 가지로 귀결됩니다. 채택, 총의 없음, 거부.  토론 종결 이후 {{제안}} 틀을 제거하고 알맞은 틀을 삽입하세요. 가령 {{정책}}, {{지침}}, {{수필}}, {{거부}}  등을 말이죠. 그 외에 다양한 틀에는 위키백과 이름공간 관련 틀 (영어 위키백과)을 참고하세요.
제안이 통과되었다면, 위키백과:사랑방에 통과된 정책의 주요 내용을 정리해서 새 제안이 정책/지침화되었음을 공지해야 합니다. 제안이 거부되었다면 특별한 합의가 있지 않는 이상 {{거부}} 틀을 지우지 않습니다. 이후 문서 개선 등을 통해 새 규칙으로 발전할 수도 있고, 다른 정책 개정 시 논란이 없는 부분에 한해 도입되기도 되기도 하니까요.
내용 변경
정책과 지침은 다른 위키백과 문서처럼 편집할 수 있습니다. 사전에 변경 사항을 논의하거나 문자화된 총의 기록를 구비할 필요는 없습니다. 그러나 정책과 지침은 민감하고 복잡합니다. 사용자들은 모든 편집 시 공동체의 관점을 충실히 반영하도록, 실수로 새로운 오류 또는 혼동의 원인이 발생하지 않도록 주의해야 합니다.' metadata={'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', '

In [3]:
# page_content 속성
splits[10].page_content


'\'제안\'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.\n좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.\n정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다. \n차후 공지가 불충분했다는 이의 제기를 피하려면, 위의 링크를 이용하여 공지하세요. 공지에 비중립적인 단어를 사용하는 등의 선전 행위는 피하세요.'

In [5]:
# metadata 속성
splits[10].metadata


{'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8',
 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전',
 'language': 'ko'}

3. 인덱싱
    분할된 텍스트를 `검색 가능한 형태`로 만드는 단계입니다. 
    인덱싱은 `검색 시간을 단축`시키고, `검색의 정확도를 높이는 데 중요한 역할`을 합니다. 
    LangChain 라이브러리를 사용하여 `텍스트를 임베딩`으로 변환하고, 이를 `저장`한 후, 저장된 임베딩을 기반으로 유사성 검색을 수행하는 과정을 보여줍니다.

    간략하게 설명하면 OpenAI의 임베딩 모델을 사용하여 텍스트를 벡터로 변환하고, 이를 Chroma 벡터저장소에 저장합니다. 
    `vectorstore.similarity_search` 메소드는 주어진 쿼리 문자열("격하 과정에 대해서 설명해주세요.")에 대해 저장된 문서들 중에서 가장 유사한 문서들을 찾아냅니다. 
    이때 유사성은 `임베딩 간의 거리`(또는 유사도)로 계산됩니다. 
    4개의 문서가 반환되는데, 가장 유사도가 높은 첫 번째 문서를 출력하여 확인합니다.

In [6]:
# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

docs = vectorstore.similarity_search("격하 과정에 대해서 설명해주세요.")
print(len(docs))
print(docs[0].page_content)


4
채택 과정
한국어 위키백과에서 오랫동안 확립되어 온 정책과 지침의 대다수는, 영어 위키백과 설립 시 토대가 된 원칙에서 발전된 것들입니다. 물론 타 언어 위키백과의 원칙을 가져오는 것 말고도, 정책과 지침을 일반적인 문제와 문서 훼손 행위의 대응책으로 한국어 위키백과 내 공동체에서 자발적으로 세우기도 했습니다. 정책과 지침 대부분은 전례 없이 바로 받아들여지기보다, 공동체의 강력한 지지를 바탕으로 세워집니다. 정책과 지침의 제정 방법으로는 제안을 통한 수립, 기존의 수필 또는 지침의 정책화, 기존의 정책과 지침의 분할 또는 합병을 통한 재구성 등의 여러 방법이 있습니다.
정책과 지침이 아닌 위키백과 내 운영과 관련된 문서에는 {{수필}}, {{정보문}}, {{위키백과 사용서}} 등을 붙여 구분해야 합니다.
현재 정책이나 지침으로 제안된 문서는 분류:위키백과 제안에 모여 있습니다. 총의를 통해 채택이 거부된 제안은 분류:위키백과 거부된 제안을 참조하세요. 여러분들의 참여를 환영합니다.
제안과 채택
 백:아님 § 관료주의  문서를 참고하십시오. 단축백:제안
제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.
'제안'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.


4. 검색
    사용자의 질문이나 주어진 컨텍스트에 가장 관련된 정보를 찾아내는 과정입니다. 
    사용자의 입력을 바탕으로 쿼리를 생성하고, 인덱싱된 데이터에서 가장 관련성 높은 정보를 검색합니다. 
    LangChain의 retriever 메소드를 사용합니다.

5. 생성
    검색된 정보를 바탕으로 사용자의 질문에 답변을 생성하는 최종 단계입니다. 
    LLM 모델에 검색 결과와 함께 사용자의 입력을 전달합니다. 
    모델은 사전 학습된 지식과 검색 결과를 결합하여 주어진 질문에 가장 적절한 답변을 생성합니다.

    검색과 생성 단계를 수행하는 다음 코드를 살펴보겠습니다. 
    vectorstore.as_retriever() 메소드는 Chroma 벡터 스토어를 검색기로 사용하여 사용자의 질문과 관련된 문서를 검색합니다. 
    format_docs 함수는 검색된 문서들을 하나의 문자열로 반환합니다. 
    RAG 체인을 구성하고, 주어진 질문에 대한 답변을 생성합니다.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# Rretriever
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke("격하 과정에 대해서 설명해주세요.")


"제공된 문맥에는 '격하 과정'에 대한 설명이 포함되어 있지 않습니다. 따라서 격하 과정에 대한 정보를 제공할 수 없습니다."

## RAG - Document Loader
LangChain에서 `Document Loader는 다양한 소스에서 문서를 불러오고 처리하는 과정을 담당`합니다. 
특히 사전지식이 필요한 지식 기반의 태스크, 정보 검색, 데이터 처리 작업 등을 처리할 때 반드시 필요합니다. 
Document Loader의 주요 목적은 `효율적으로 문서 데이터를 수집하고, 사용 가능한 형식으로 변환`하는 것입니다.

- 다양한 소스 지원
  웹 페이지, PDF 파일, 데이터베이스 등 다양한 소스에서 문서를 불러올 수 있습니다.

- 데이터 변환 및 정제
  `불러온 문서 데이터를 분석하고 처리하여, 랭체인의 다른 모듈이나 알고리즘이 처리하기 쉬운 형태로 변환`합니다. 
  불필요한 데이터를 제거하거나, 구조를 변경할 수도 있습니다.

- 효율적인 데이터 관리
  대량의 문서 데이터를 효율적으로 관리하고, 필요할 때 쉽게 접근할 수 있도록 합니다. 이를 통해 검색 속도를 향상시키고, 전체 시스템의 성능을 높일 수 있습니다.

사용 예시:
웹 크롤링을 통해 특정 주제에 관한 기사나 논문을 자동으로 수집하고, 이를 분석하여 요약 정보를 생성하는 애플리케이션.
기업 내부 문서 저장소에서 필요한 문서를 빠르게 검색하고, 관련 정보를 추출하여 보고서를 자동으로 작성하는 시스템

In [16]:
# 웹 문서
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 여러 개의 url 지정 가능
url1 = "https://blog.langchain.dev/customers-replit/"
url2 = "https://blog.langchain.dev/langgraph-v0-2/"

loader = WebBaseLoader(
    web_paths=(url1, url2),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("article-header", "article-content")
        )
    ),
)
docs = loader.load()
len(docs)
print(docs[0].page_content)
print(docs[0].metadata)
print("--------------------------------")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[10])



Replit is at the forefront of AI innovation with its platform that simplifies writing, running, and collaborating on code for over 30+ million developers. They recently released Replit Agent, which immediately went viral due to the incredible applications people could easily create with this tool.Behind the scenes, Replit Agent has a complex workflow which enables a highly custom agentic workflow with a high-degree of control and parallel execution. By using LangSmith, Replit gained deep visibility into their agent interactions to debug tricky issues. The level of complexity required for Replit Agent also pushed the boundaries of LangSmith. The LangChain and Replit teams worked closely together to add functionality to LangSmith that would satisfy their LLM observability needs. Specifically, there were three main areas that we innovated on:Improved performance and scale on large tracesAbility to search and filter within tracesThread view to enable human-in-the loop workflowsImproved pe

In [23]:
# 디렉토리 폴더(Directory Loader)
import os
from glob import glob

files = glob(os.path.join('./', '*.docx'))
files


['./내부분석환경_사용자매뉴얼(수정).docx']

In [48]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, Docx2txtLoader
# pdf 로더
# pip installpypdf
# docx 로더
# pip install docx2txt
loader = DirectoryLoader(path='./', glob='*.docx', loader_cls=Docx2txtLoader)

docs = loader.load()

len(docs)
# print(docs)
# print(docs[0].page_content)
# print(docs[0].metadata)

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

# print(len(splits))
# print(splits[0])

# import re

# def clean_ws(text: str) -> str:
#     text = text.replace("\u00a0", " ")
#     return re.sub(r"\s+", " ", text).strip()

# for d in docs:
#     d.page_content = clean_ws(d.page_content)

# print(docs[0].page_content)
# print(docs[0].metadata)

import re

def keep_newlines_reduce_blank_lines(text: str) -> str:
    # 특수 공백/개행 정리
    text = text.replace("\u00a0", " ")      # NBSP -> 일반 공백
    text = text.replace("\r\n", "\n").replace("\r", "\n")

    # 빈 줄 판정에 방해되는 "줄 끝 공백" 제거
    text = re.sub(r"[ \t]+\n", "\n", text)

    # (핵심) 2개 이상 연속 빈 줄 -> "빈 줄 1개"로 축소
    # 즉, \n\n\n... -> \n\n
    text = re.sub(r"\n{3,}", "\n\n", text)

    return text.strip()

for d in docs:
    d.page_content = keep_newlines_reduce_blank_lines(d.page_content)

# 이제 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# print(splits[3].page_content)

# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

# docs = vectorstore.similarity_search("분석환경 생성 과정에 대해서 설명해주세요.")
# print(len(docs))
# print(docs[0].page_content)

template = '''질문에 따라 답변해주세요:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# Rretriever
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke("분석환경 생성 과정에 대해서 설명해주세요.")

'분석환경 생성 과정은 다음과 같습니다:\n\n### 1. 개인 분석환경 생성\n1. **리소스 요청**: 분석과제에 소속된 사용자는 과제 관리자에게 할당받은 리소스를 기반으로 분석환경을 생성할 수 있습니다. 필요한 리소스를 요청하기 위해 [리소스 요청] 버튼을 클릭합니다.\n2. **정보 입력**: 요청할 리소스의 정보를 입력합니다. 이때 CPU는 최소 2개, RAM은 최소 4GB 이상이어야 합니다.\n3. **환경 추가**: [새 환경 추가+] 버튼을 클릭하여 개인 분석환경을 추가합니다.\n4. **분석환경 추가**: 정보를 입력한 후 [분석환경 추가] 버튼을 클릭하여 환경을 생성합니다.\n5. **확인**: 생성된 분석환경은 분석환경 페이지에서 확인할 수 있습니다. (분석환경 > 개인분석환경)\n6. **사용 및 수정**: 생성된 분석환경은 자유롭게 사용할 수 있으며, 이후 수정도 가능합니다. 단, 이미지 태그에 한정됩니다.\n7. **삭제 및 재생성**: 필요 시 생성된 분석환경을 삭제하여 리소스를 회수하고 새로운 분석환경을 생성할 수 있습니다.\n\n### 2. 공용 분석환경 생성\n1. **리소스 요청**: 분석과제에 소속된 사용자는 사전에 생성된 Jupyter Hub의 접속 URL을 등록하여 공용 분석환경을 생성할 수 있습니다.\n2. **정보 입력**: [새 환경 추가+] 버튼을 클릭하여 공용 분석환경을 추가합니다.\n3. **환경 추가**: 정보를 입력한 후 [분석환경 추가] 버튼을 클릭하여 등록합니다.\n4. **확인**: 등록한 분석환경은 분석환경 페이지에서 확인할 수 있습니다. (분석환경 > 공용분석환경)\n5. **사용 및 수정**: 생성된 공용 분석환경은 자유롭게 사용할 수 있으며, 이후 수정도 가능합니다. 단, 소속 과제는 고정됩니다.\n\n### 3. Inference 영역 분석환경 생성\n1. **IT 관리자 요청**: Inference 영역의 분석환경은 IT 관리자에게 생성을 요청해야 합니다.\n2. **기본 사양**: 기본 값

## RAG -Text Splitter
LangChain은 긴 문서를 작은 단위인 청크(chunk)로 나누는 텍스트 분리 도구를 다양하게 지원합니다. 
텍스트를 분리하는 작업을 `청킹(chunking)`이라고 부르기도 합니다. 
이렇게 문서를 작은 조각으로 나누는 이유는 `LLM 모델의 입력 토큰의 개수가 정해져 있기 `때문입니다. 
허용 한도를 넘는 텍스트는 모델에서 입력으로 처리할 수 없게 되는 것입니다. 
한편, 텍스트가 너무 긴 경우에는 핵심 정보 이외에 불필요한 정보들이 많이 포함될 수 있어서 RAG 품질이 낮아지는 요인이 될 수도 있습니다. 
핵심 정보가 유지될 수 있는 적절한 크기로 나누는 것이 매우 중요합니다.

LangChain이 지원하는 다양한 텍스트 분리기(Text Splitter)는 분할하려는 텍스트 유형과 사용 사례에 맞춰 선택할 수 있는 다양한 옵션이 제공됩니다. 크게 두 가지 차원에서 검토가 필요합니다.

1. 텍스트가 어떻게 분리되는지:
    텍스트를 나눌 때 각 청크가 독립적으로 의미를 갖도록 나눠야 합니다. 
    이를 위해 문장, 구절, 단락 등 문서 구조를 기준으로 나눌 수 있습니다.

2. 청크 크기가 어떻게 측정되는지:
    각 청크의 크기를 직접 조정할 수 있습니다. 
    LLM 모델의 입력 크기와 비용 등을 종합적으로 고려하여 애플리케이션에 적합한 최적 크기를 결정하는 기준입니다. 
    예를 들면 단어 수, 문자 수 등을 기준으로 나눌 수 있습니다.

<details>
<summary>LLM 모델의 입력 토큰 수 제한이란?</summary>

> LLM 모델은 **한 번에 처리할 수 있는 전체 입력의 크기**에 제한이 있습니다. 이 입력에는:
> 
> 1. **시스템 프롬프트** (모델에게 주는 지시사항)
> 2. **사용자 질문** (당신이 물어보는 내용)
> 3. **문맥 정보** (RAG에서 검색된 문서 내용) ← **이 부분이 핵심!**
> 
> 이 **모든 것을 합친 총량**이 모델의 토큰 제한을 넘으면 안 됩니다.
> 
> ## 구체적인 예시
> 
> 예를 들어, GPT-4 모델의 토큰 제한이 8,000토큰이라고 가정하면:
> 
> ```
> [시스템 프롬프트] 500토큰
> [사용자 질문] "이 회사의 2023년 매출은?" 50토큰
> [검색된 문서] 100페이지짜리 연간보고서 전체 → 50,000토큰 ❌
> ```
> 
> → **문제 발생!** 전체 문서를 넣으면 토큰 제한 초과
> 
> ## RAG에서 Text Splitter가 필요한 이유
> 
> 그래서 RAG는 이렇게 작동합니다:
> 
> 1. **사전에 긴 문서를 작은 청크로 분할** (Text Splitter 사용)
>    - 100페이지 보고서 → 500개의 작은 청크로 분할
>    
> 2. **사용자가 질문하면**
>    - 질문과 관련된 청크만 3~5개 정도 검색
>    
> 3. **LLM에 입력**
>    ```
>    [시스템 프롬프트] 500토큰
>    [사용자 질문] 50토큰
>    [검색된 관련 청크 5개] 2,000토큰 ✅
>    총 2,550토큰 → 제한 내!
>    ```
> 
> ## 핵심 포인트
> 
> - **"입력 토큰 수"는 질문만이 아니라, 질문 + 검색된 문서 내용 + 프롬프트를 모두 합친 것**
> - RAG에서는 긴 문서를 통째로 넣는 게 아니라, 질문과 관련된 작은 조각들만 선별해서 넣음
> - 그래서 **미리** 문서를 작은 청크로 나눠두는 것 (청킹)

</deatils>

In [ ]:
# CharacterTextSplitter
# TextLoader 클래스는 특정 파일에서 텍스트를 로드해서 Document 객체로 변환합니다. 
# 여기서는 'history.txt' 파일에서 텍스트를 로드하고, 로드된 데이터 중에서 첫 번째 Document 객체의 페이지 내용의 길이와 해당 내용을 출력하고 있습니다. 
# 글자 수는 총 1,234개로 확인됩니다.
from langchain_community.document_loaders import TextLoader

loader = TextLoader('history.txt')
data = loader.load()

print(len(data[0].page_content))
data[0].page_content

1. 문서를 개별 문자를 단위로 나누기 (separator="")
CharacterTextSplitter 클래스는 주어진 텍스트를 문자 단위로 분할하는 데 사용됩니다. 
파이썬의 split 함수라고 생각하시면 됩니다. 
다음 코드에서 적용된 주요 매개변수는 다음과 같습니다:

`separator` : `분할된 각 청크를 구분할 때 기준이 되는 문자열`입니다. 여기서는 빈 문자열('')을 사용하므로, 각 글자를 기준으로 분할합니다.
`chunk_size` : 각 청크의 최대 길이입니다. 여기서는 500으로 설정되어 있으므로, 최대 500자까지의 텍스트가 하나의 청크에 포함됩니다.
`chunk_overlap` : 인접한 청크 사이에 중복으로 포함될 문자의 수입니다. 여기서는 100으로 설정되어 있으므로, 각 청크들은 연결 부분에서 100자가 중복됩니다.
`length_function` : 청크의 길이를 계산하는 함수입니다. 여기서는 len 함수가 사용되었으므로, 문자열의 길이를 기반으로 청크의 길이를 계산합니다.

In [56]:
# 각 문자를 구분하여 분할
from langchain_text_splitters import CharacterTextSplitter

print(len(data[0].page_content))
print(len(data[0].page_content)/400)
print((len(data[0].page_content)-500)/400)
text_splitter = CharacterTextSplitter(
    separator = '',
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.split_text(data[0].page_content)
# split_text 메소드는 주어진 텍스트(data[0].page_content)를 위에서 설정한 매개변수에 따라 분할하고, 분할된 청크의 리스트를 반환합니다. 
print(len(texts))
print(len(texts[0]), len(texts[1]), len(texts[2]))
# len(texts)는 분할된 청크의 총 수를 나타냅니다. 여기서는 3개의 청크로 분할됩니다.

# 여기서 중요한 것은 각 청크의 크기가 chunk_size를 초과하지 않으며, 인접한 청크 사이에는 chunk_overlap만큼의 문자가 중복되어 있음을 이해하는 것입니다. 
# 이렇게 함으로써, 텍스트의 의미적 연속성을 유지하면서도 큰 데이터를 더 작은 단위로 분할할 수 있습니다.

13345
33.3625
32.1125
34
499 493 497


2. 문서를 특정 문자열을 기준으로 나누기 (separator="문자열")
CharacterTextSplitter 클래스의 separator 매개변수를 줄바꿈 문자('\n')로 설정하는 예제입니다. 
이렇게 하면 각 청크를 나누는 기준을 줄바꿈 문자로 설정하는 것입니다.

In [55]:
# 줄바꿈 문자를 기준으로 분할

text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.split_text(data[0].page_content)

print(len(texts))
print(len(texts[0]), len(texts[1]), len(texts[2]))



32
485 478 500


### RecursiveCharacterTextSplitter
`RecursiveCharacterTextSplitter` 클래스는 `텍스트를 재귀적으로 분할하여 의미적으로 관련 있는 텍스트 조각들이 같이 있도록 하는 목적`으로 설계되었습니다. 
이 과정에서 문자 리스트(['\n\n', '\n', ' ', ''])의 문자를 순서대로 사용하여 텍스트를 분할하며, 분할된 청크들이 설정된 chunk_size보다 작아질 때까지 이 과정을 반복합니다. 
여기서 chunk_overlap은 분할된 텍스트 조각들 사이에서 중복으로 포함될 문자 수를 정의합니다. 
length_function = len 코드는 분할의 기준이 되는 길이를 측정하는 함수로 문자열의 길이를 반환하는 len 함수를 사용한다는 의미입니다.

In [57]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.split_text(data[0].page_content)

len(texts)
# texts = text_splitter.split_text(data[0].page_content) 코드는 data[0].page_content에서 첫 번째 문서의 내용을 RecursiveCharacterTextSplitter를 사용하여 분할하고, 결과를 texts 변수에 할당합니다. 
# data 리스트에서 첫 번째 문서의 내용을 기반으로 분할 작업을 수행하게 됩니다. 
# len(texts)는 분할된 텍스트 조각들의 총 수를 반환합니다.

34

### 토큰 수를 기준으로 텍스트 분할 (Tokenizer 활용)
대규모 언어 모델(LLM)을 사용할 때 모델이 처리할 수 있는 토큰 수에는 한계가 있습니다. 
입력 데이터를 모델의 제한을 초과하지 않도록 적절히 분할하는 것이 중요합니다. 
이때 LLM 모델에 적용되는 토크나이저를 기준으로 텍스트를 토큰으로 분할하고, 이 토큰들의 수를 기준으로 텍스트를 청크로 나누면 모델 입력 토큰 수를 조절할 수 있습니다.

In [58]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=600,
    chunk_overlap=200,
    encoding_name='cl100k_base'
)

docs = text_splitter.split_documents(data)
len(docs)

#OpenAI API의 경우 tiktoken 라이브러리를 통해 해당 모델에서 사용하는 토크나이저를 기준으로 분할할 수 있습니다. 
# CharacterTextSplitter.from_tiktoken_encoder 메서드는 글자 수 기준으로 분할할 때 tiktoken 토크나이저를 기준으로 글자 수를 계산하여 분할합니다. 
# 여기서 encoding_name='cl100k_base'는 텍스트를 토큰으로 변환하는 인코딩 방식을 나타냅니다.

23

## RAG - Embedding
임베딩(Embedding)은 텍스트 데이터를 숫자로 이루어진 벡터로 변환하는 과정을 말합니다. 
이러한 벡터 표현을 사용하면, `텍스트 데이터를 벡터 공간 내에서 수학적으로 다룰 수 있게 되며, 이를 통해 텍스트 간의 유사성을 계산하거나, 텍스트 데이터를 기반으로 하는 다양한 머신러닝 및 자연어 처리 작업을 수행`할 수 있습니다. 
임베딩 과정은 텍스트의 의미적인 정보를 보존하도록 설계되어 있어, 벡터 공간에서 가까이 위치한 텍스트 조각들은 의미적으로도 유사한 것으로 간주됩니다.

- 임베딩의 주요 활용 사례:
    `의미 검색(Semantic Search)`
        벡터 표현을 활용하여 의미적으로 유사한 텍스트를 검색하는 과정으로, 사용자가 입력한 쿼리에 대해 가장 관련성 높은 문서나 정보를 찾아내는 데 사용됩니다.
    `문서 분류(Document Classification)`
        임베딩된 텍스트 벡터를 사용하여 문서를 특정 카테고리나 주제에 할당하는 분류 작업에 사용됩니다.
    `텍스트 유사도 계산(Text Similarity Calculation)`
        두 텍스트 벡터 사이의 거리를 계산하여, 텍스트 간의 유사성 정도를 정량적으로 평가합니다.
- 임베딩 모델 제공자:
    `OpenAI`
        GPT와 같은 언어 모델을 통해 텍스트의 임베딩 벡터를 생성할 수 있는 API를 제공합니다.
    `Hugging Face`
        Transformers 라이브러리를 통해 다양한 오픈소스 임베딩 모델을 제공합니다.
    `Google`
        Gemini, Gemma 등 언어 모델에 적용되는 임베딩 모델을 제공합니다.
- 임베딩 메소드:
    `embed_documents`
        이 메소드는 문서 객체의 집합을 입력으로 받아, 각 문서를 벡터 공간에 임베딩합니다. 
        주로 대량의 텍스트 데이터를 배치 단위로 처리할 때 사용됩니다.
    `embed_query`
        이 메소드는 단일 텍스트 쿼리를 입력으로 받아, 쿼리를 벡터 공간에 임베딩합니다. 
        주로 사용자의 검색 쿼리를 임베딩하여, 문서 집합 내에서 해당 쿼리와 유사한 내용을 찾아내는 데 사용됩니다.

임베딩은 텍스트 데이터를 머신러닝 모델이 이해할 수 있는 형태로 변환하는 핵심 과정입니다.
다양한 자연어 처리 작업의 기반이 되는 중요한 작업입니다.

### OpenAIEmbeddings
`OpenAIEmbeddings` 클래스는 OpenAI의 API를 활용하여, 각 문서를 대응하는 임베딩 벡터로 변환합니다. 
langchain_openai 라이브러리에서 OpenAIEmbeddings 클래스를 직접 임포트합니다.

In [59]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [60]:
embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

`embed_documents` 메소드는 입력 받은 5개의 문서 객체를 각각 별도의 벡터로 임베딩합니다.
`embeddings` 변수에는 각 텍스트에 대한 벡터 표현을 담고 있는 리스트가 할당됩니다. 
len(embeddings)는 입력된 텍스트 리스트의 개수와 동일하며, 이는 임베딩 과정을 거친 문서의 총 수를 나타냅니다.

len(embeddings[0])는 첫 번째 문서의 벡터 표현의 차원을 나타냅니다. 
일반적으로 이 차원 수는 선택된 모델에 따라 정해지며, 모든 임베딩 벡터는 동일한 차원을 가집니다. 
OpenAI의 임베딩 모델을 사용할 경우 임베딩 벡터의 차원은 1536이라는 것을 확인할 수 있습니다.

In [61]:
print(embeddings[0])

[-0.01041258405894041, -0.01355851348489523, -0.006538722664117813, -0.018673023208975792, -0.018280573189258575, 0.016685454174876213, -0.009216244332492352, 0.003937159199267626, -0.0074185701087117195, 0.0100644426420331, 0.011760839261114597, -0.006709628272801638, -0.02540796995162964, -0.02252156473696232, -0.004892964847385883, -0.021761983633041382, 0.025281373411417007, -0.01764758862555027, 0.00793128740042448, -0.017837483435869217, -0.008038894273340702, -0.01931866630911827, 0.005560762714594603, -0.008747836574912071, 0.0007754051475785673, 0.007937616668641567, 0.0012129552196711302, -0.017698228359222412, 0.017964079976081848, -0.028231078758835793, 0.012134299613535404, 0.006212736014276743, -0.015444804914295673, 0.000377218791982159, 0.009545396082103252, -0.003971973434090614, 0.0018815443618223071, -0.009456777945160866, 0.0034845760092139244, -0.003680800786241889, 0.009804919362068176, -0.016406940296292305, 0.004165033344179392, -0.017698228359222412, -0.0032820

In [62]:
embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')
embedded_query[:5]

[0.003640108974650502,
 -0.024275783449411392,
 0.010910888202488422,
 -0.04110145568847656,
 -0.004543057177215815]

`embed_query` 메소드는 단일 쿼리 문자열을 받아 이를 벡터 공간에 임베딩합니다. 
주로 검색 쿼리나 질문 같은 단일 텍스트를 임베딩할 때 유용하며, 생성된 임베딩을 사용해 유사한 문서나 답변을 찾을 수 있습니다.

embedded_query[:5]는 생성된 임베딩 벡터의 처음 5개 원소를 슬라이싱하여 반환합니다. 
임베딩의 일부 특성을 살펴볼 수 있습니다.

In [63]:
# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))


0.8347793912001632
0.8154197762848945
0.8844172747319565
0.7898703827307417
0.7467077657972327


코사인 유사도는 두 벡터 간의 코사인 각을 이용하여 유사성을 측정하는 방법입니다. 
두 벡터의 방향이 완전히 동일하면 코사인 유사도는 1이 됩니다. 
90도로 수직이면 0, 반대 방향이면 -1이 됩니다. 이는 텍스트 임베딩과 같이 고차원 공간에서 벡터 간 유사도를 측정하는 데 유용하게 사용됩니다.

주어진 cos_sim 함수는 두 벡터 A와 B 사이의 코사인 유사도를 계산합니다. 
dot(A, B)는 두 벡터의 내적을, norm(A)와 norm(B)는 각각 벡터 A와 B의 노름(크기)을 계산합니다. 
이 함수는 내적 값과 두 벡터 크기의 곱으로 나눈 값으로 코사인 유사도를 계산합니다.

이 예시에서는 앞에서 임베딩 변환한 문서들(embeddings)과 하나의 임베딩된 쿼리(embedded_query) 사이의 코사인 유사도를 계산하여 출력합니다. 
각 문서 임베딩에 대해 cos_sim 함수를 호출하여, 해당 문서가 쿼리와 얼마나 유사한지를 숫자로 나타냅니다. 유사도가 높은 문서일수록 쿼리와 더 관련이 깊다고 볼 수 있습니다.

### HuggingFaceEmbeddings
`sentence-transformers` 라이브러리를 사용하면 HuggingFace 모델에서 사용된 사전 훈련된 임베딩 모델을 다운로드 받아서 적용할 수 있습니다. OpenAI 임베딩 모델을 사용할 때는 API 사용료가 부과되지만, HuggingFace의 오픈소스 기반의 임베딩 모델을 사용하면 요금이 부과되지 않습니다.

sentence-transformers 라이브러리를 설치
```
pip install -U sentence-transformers

```

HuggingFaceEmbeddings 클래스는 Hugging Face의 트랜스포머 모델을 사용하여 문서 또는 문장을 임베딩하는 데 사용됩니다. 
다음은 주요 매개변수의 설정 값을 설명합니다.

- `model_name='jhgan/ko-sroberta-nli'`
  - 사용할 모델을 지정합니다. 
  - 여기서는 한국어 자연어 추론(Natural Language Inference, NLI)에 최적화된 ko-sroberta 모델을 사용합니다.
- `model_kwargs={'device':'cpu'}`
  - 모델이 CPU에서 실행되도록 설정합니다. 
  - GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있습니다.
- `encode_kwargs={'normalize_embeddings':True}`
  - 임베딩을 정규화하여 모든 벡터가 같은 범위의 값을 갖도록 합니다. 
  - 이는 유사도 계산 시 일관성을 높여줍니다.


In [64]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

embeddings_model

# embeddings_model 을 출력해보면 Pooling 레이어의 word_embedding_dimension 값에서 임베딩 벡터의 크기를 확인할 수 있습니다. 
# 768차원의 벡터라는 것을 알 수 있습니다.

/var/folders/d3/h50bjcxd1nxb6d98lbh8jhgw0000gn/T/ipykernel_289/1536508419.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
/Users/seohyuktaek/Documents/study/langchain/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'RobertaModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='jhgan/ko-sroberta-nli', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [65]:
embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)
len(embeddings), len(embeddings[0])


(5, 768)

In [66]:
embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))


0.5899015507255911
0.4182630956904064
0.7240603990052181
0.057026526289353854
0.43164171480259006


<details>
<summary>차원수</summary>

> 차원이 다르다고 해서 “1536이 무조건 더 좋다”거나 “768은 나쁘다”는 뜻은 아닙니다. **임베딩 차원(dimension)은 “문장을 숫자 벡터로 표현할 때, 그 벡터가 몇 개의 숫자로 이루어져 있느냐”**를 말해요.
> 
> ---
> 
> ## 1) 여기서 말하는 “차원”이 뭐야?
> 
> 임베딩은 문장/문서를 다음처럼 **벡터(숫자 배열)** 로 바꾼 결과입니다.
> 
> - `ko-sroberta-nli` → 각 문장을 **768개 숫자**로 표현 (벡터 길이 768)
> - `OpenAIEmbeddings` → 각 문장을 **1536개 숫자**로 표현 (벡터 길이 1536)
> 
> 즉,
> - (5, 768)은 “문장 5개, 각 문장은 768차원 벡터”
> - (5, 1536)은 “문장 5개, 각 문장은 1536차원 벡터”
> 
> 이 차원 벡터는 RAG에서 주로 **유사도 계산(코사인 유사도 등)** 에 쓰입니다.  
> “질문 벡터”와 “문서 청크 벡터”가 가까우면 관련성이 높다고 보고 가져오는 거죠.
> 
> ---
> 
> ## 2) 그럼 차원이 크면 더 좋은 거 아니야?
> 
> 꼭 그렇지 않습니다. 품질은 단순히 차원 수로 결정되지 않고, 아래가 더 중요합니다.
> 
> ### 더 중요한 요소들
> 1. **모델이 어떤 데이터/목표로 학습되었는지**  
>    - 한국어에 강한지
>    - “검색용(retrieval)”으로 학습된 모델인지
>    - 문장 유사도/의미검색에 특화됐는지
> 
> 2. **도메인 적합성**
>    - 네 문서가 법률/의학/코드/사내문서/대화체 등 무엇인지에 따라 달라짐
> 
> 3. **정규화(normalize_embeddings) / distance metric / 인덱스 설정**
>    - normalize를 켜면 코사인 유사도 기반 검색에서 안정적인 경우가 많음(지금 설정은 좋은 편)
> 
> 4. **Chunking / Rerank / Hybrid 검색**  
>    - 실제 RAG 품질은 임베딩 모델 단독보다 파이프라인 전체 영향이 큼
> 
> ---
> 
> ## 3) 실무적으로 “무엇이 더 좋냐”는 이렇게 판단해
> 
> ### A. 한국어 RAG(온프렘)라면
> - `jhgan/ko-sroberta-nli`는 **한국어 문장 의미 유사도** 용도로는 괜찮을 수 있지만,
> - RAG “검색”에서는 **retrieval 최적화된 임베딩 모델**이 더 유리한 경우가 많습니다(예: bge 계열, e5 계열 등).
> 
> 즉, “OpenAI(1536) vs ko-sroberta(768)”의 승부가 아니라  
> **“검색용으로 강한 임베딩 모델인가?”**가 핵심이에요.
> 
> ### B. OpenAIEmbeddings가 유리한 경우(일반적으로)
> - 다양한 언어/도메인에서 평균적으로 성능이 잘 나오는 편
> - 다만 **온프렘 목표**라면 비용/프라이버시/네트워크 의존성이 단점
> 
> ### C. 차원 관점에서의 트레이드오프
> - **차원↑(1536)**: 표현력 잠재력↑ 가능 / 하지만 저장공간↑, 검색 약간 느려질 수 있음
> - **차원↓(768)**: 가볍고 빠름 / 하지만 모델 자체 성능이 더 중요
> 
> ---
> 
> ## 4) “같은 벡터DB에 768과 1536을 섞어도 되나?”
> 안 됩니다. 벡터DB에서 유사도 계산은 **차원이 동일해야** 해요.
> 
> - 인덱스(컬렉션) 하나에는 보통 **하나의 임베딩 모델(=고정 차원)** 만 사용합니다.
> - 모델을 바꾸면 보통 **전체 문서를 재임베딩 후 재인덱싱** 해야 합니다.
> 
> ---
> 
> ## 5) 결론: 너는 뭘 쓰는 게 좋아?
> 
> 당신이 “온프렘 LLM + 온프렘 RAG”를 하려는 목적이라면:
> 
> - 임베딩도 온프렘으로 가는 게 자연스럽고,
> - 그 경우 “한국어 검색”에 강한 **retrieval 특화 임베딩 모델**을 추천하는 쪽이 보통 결과가 좋습니다.
> - 단순히 **768 vs 1536**로 고르는 게 아니라, **네 문서/질문에서 검색 정확도를 측정**해서 고르는 게 정답입니다.
> 
> ---
> 
> ## 6) 내가 바로 추천/가이드해줄 수 있게 3가지만 알려줘
> 1) 문서 종류: PDF 위주? (논문/매뉴얼/사내규정/코드/대화로그 등)  
> 2) 질문 형태: “요약/정의/근거찾기/QA” 중 어떤 게 많아?  
> 3) 집 PC에 GPU 있어? 있다면 VRAM 몇 GB?
> 
> 이거 주면 “온프렘 임베딩 모델 후보 2~3개 + 청킹값 + 평가 방법(작게 테스트셋 만들어서 비교)”까지 딱 맞춰서 제안해줄게.

</details>

## RAG - Vector Store
벡터 저장소(Vector Store)는 벡터 형태로 표현된 데이터, 즉 임베딩 벡터들을 효율적으로 저장하고 검색할 수 있는 시스템이나 데이터베이스를 의미합니다. 자연어 처리(NLP), 이미지 처리, 그리고 기타 다양한 머신러닝 응용 분야에서 생성된 고차원 벡터 데이터를 관리하기 위해 설계되었습니다. 
벡터 저장소의 핵심 기능은 대규모 벡터 데이터셋에서 빠른 속도로 가장 유사한 항목을 찾아내는 것입니다.

### Chroma
Chroma는 임베딩 벡터를 저장하기 위한 오픈소스 소프트웨어로, LLM(대규모 언어 모델) 앱 구축을 용이하게 하는 핵심 기능을 수행합니다. 
Chroma의 주요 특징은 다음과 같습니다:

- 임베딩 및 메타데이터 저장
  - 대규모의 임베딩 데이터와 이와 관련된 메타데이터를 효율적으로 저장할 수 있습니다.
- 문서 및 쿼리 임베딩
  - 텍스트 데이터를 벡터 공간에 매핑하여 임베딩을 생성할 수 있으며, 이를 통해 검색 작업이 가능합니다.
- 임베딩 검색
  - 사용자 쿼리에 기반하여 가장 관련성 높은 임베딩을 찾아내는 검색 기능을 제공합니다.

#### 유사도 기반 검색
`Chroma` 벡터 저장소를 사용하여 임베딩된 텍스트 데이터를 저장하고 검색하는 방법을 설명합니다. 
Chroma 벡터 저장소를 사용하여 대규모 텍스트 데이터셋에서 빠르고 효율적으로 유사도 기반 검색(Similarity search)을 수행할 수 있습니다. 
구체적인 단계는 다음과 같습니다

1. 텍스트 데이터 로드
2. 텍스트 분할
3. 임베딩 모델 초기화
4. Chroma 벡터 저장소 생성
5. 유사도 기반 검색 수행

In [71]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = TextLoader('history.txt')
#TextLoader 클래스를 사용해 history.txt 파일에서 텍스트 데이터를 로드합니다.

data = loader.load()
# 로드된 데이터는 data 변수에 저장됩니다.

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=50,
    encoding_name='cl100k_base'
)
# RecursiveCharacterTextSplitter를 사용하여 로드된 텍스트를 여러 개의 작은 조각으로 분할합니다

texts = text_splitter.split_text(data[0].page_content)
# 분할된 텍스트는 texts 변수에 저장됩니다.

texts[0]
# 분할된 텍스트의 첫 번째 조각을 출력합니다.

embeddings_model = OpenAIEmbeddings()
# OpenAIEmbeddings 모델을 초기화합니다.
# OpenAIEmbeddings를 사용하여 OpenAI 임베딩 모델의 인스턴스를 생성합니다. 
# 이 단계에서 Huggingface 또는 다른 임베딩 모델을 사용할 수 있습니다.

db = Chroma.from_texts(
    texts, 
    embeddings_model,
    collection_name = 'history',
    # 저장소는 collection_name으로 구분되며, 여기서는 'history'라는 이름을 사용합니다.
    persist_directory = './db/chromadb',
    # 저장된 데이터는 ./db/chromadb 디렉토리에 저장됩니다.
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
    # collection_metadata에서 'hnsw:space': 'cosine'을 설정하여 유사도 계산에 코사인 유사도를 사용합니다.
)

# Chroma 벡터 저장소를 생성합니다.
# Chroma.from_texts 메소드를 사용하여 분할된 텍스트들을 임베딩하고, 이 임베딩을 Chroma 벡터 저장소에 저장합니다.


db

# 생성된 Chroma 벡터 저장소를 출력합니다.

query = '누가 한글을 창제했나요?'
# query 변수에 검색 쿼리를 정의합니다.

docs = db.similarity_search(query)
# db.similarity_search 메소드를 사용하여 저장된 데이터 중에서 쿼리와 가장 유사한 문서를 찾습니다.

print(docs[0].page_content)

# 유사도 기반 검색을 수행합니다.
# 검색 결과를 docs 변수에 저장하고, 가장 유사한 문서의 내용은 docs[0].page_content를 통해 확인합니다.
# 이 과정을 통해, 주어진 쿼리('누가 한글을 창제했나요?')에 대해 가장 관련성 높은 텍스트 조각('...세종대왕이 한글을 창제하여...')을 찾아내고 있습니다.


조선은 1392년 이성계에 의해 건국되어, 1910년까지 이어졌습니다. 조선 초기에는 세종대왕이 한글을 창제하여 백성들의 문해율을 높이는 등 문화적, 과학적 성취가 이루어졌습니다. 그러나 조선 후기에는 내부적으로 실학의 발전과 함께 사회적 변화가 모색되었으나, 외부로부터의 압력은 점차 커져만 갔습니다.


#### MMR
최대 한계 관련성(Maximum Marginal Relevance, MMR) 검색 방식은 유사성과 다양성의 균형을 맞추어 검색 결과의 품질을 향상시키는 알고리즘입니다. 이 방식은 검색 쿼리에 대한 문서들의 관련성을 최대화하는 동시에, 검색된 문서들 사이의 중복성을 최소화하여, 사용자에게 다양하고 풍부한 정보를 제공하는 것을 목표로 합니다

- MMR의 작동 원리
  - MMR은 쿼리에 대한 각 문서의 유사성 점수와 이미 선택된 문서들과의 다양성(또는 차별성) 점수를 조합하여, 각 문서의 최종 점수를 계산합니다. 이 최종 점수에 기반하여 문서를 선택합니다. MMR은 다음과 같이 정의될 수 있습니다
  ![MMR 계산식](/Users/seohyuktaek/Documents/study/langchain/SCR-20260123-mokm.png)
- MMR의 주요 매개변수
  - `query`
    - 사용자로부터 입력받은 검색 쿼리입니다.
  - `k`
    - 최종적으로 선택할 문서의 수입니다. 
    - 이 매개변수는 반환할 문서의 총 개수를 결정합니다.
  - `fetch_k`
    - MMR 알고리즘을 수행할 때 고려할 상위 문서의 수입니다. 
    - 이는 초기 후보 문서 집합의 크기를 의미하며, 이 중에서 MMR에 의해 최종 문서가 k개 만큼 선택됩니다.
  - `lambda_mult`
    - 쿼리와의 유사성과 선택된 문서 간의 다양성 사이의 균형을 조절합니다.

MMR 방식을 사용하면, 검색 결과로 얻은 문서들이 쿼리와 관련성이 높으면서도 서로 다른 측면이나 정보를 제공하도록 할 수 있습니다. 
이는 특히 정보 검색이나 추천 시스템에서 사용자에게 더 풍부하고 만족스러운 결과를 제공하는 데 도움이 됩니다.



In [78]:
# Load data -> Text split

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = PyMuPDFLoader('73da18d672b84bb06ac7e44e47ac48dc.pdf')
# PyMuPDFLoader를 사용하여 PDF 파일('323410_카카오뱅크_2023.pdf')에서 텍스트 데이터를 로드합니다. 
# 이 클래스는 PyMuPDF 라이브러리를 사용하여 PDF 문서의 내용을 추출합니다. 
# 필요한 경우 pip install pymupdf 명령어로 라이브러리를 설치합니다.
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name='cl100k_base'
)
# RecursiveCharacterTextSplitter를 사용하여 문서를 텍스트 조각으로 분할하는 인스턴스를 생성하고 text_splitter.split_documents(data)를 호출하여 로드된 문서 객체를 여러 개의 청크로 분할합니다. 
# documents 변수에는 모두 145개의 문서 조각으로 분할되어 저장됩니다
documents = text_splitter.split_documents(data)
len(documents)

183

In [79]:
# Embedding -> Upload to Vectorstore
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()
# OpenAIEmbeddings 클래스를 사용하여 임베딩 모델의 인스턴스를 생성합니다. 

db2 = Chroma.from_documents(
    documents, 
    embeddings_model,
    collection_name = 'esg',
    # 여기서는 esg라는 컬렉션 이름을 사용하며, 데이터는 ./db/chromadb 디렉토리에 저장됩니다. 
    persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
    # collection_metadata를 통해 유사도 검색에 사용될 공간('hnsw:space')을 'cosine'으로 지정하여, 코사인 유사도를 사용합니다.
)
# Chroma.from_documents 메소드를 사용하여 분할된 문서들을 임베딩하고, 이 임베딩들을 Chroma 벡터 저장소에 저장합니다. 
db2

이 과정을 통해, PDF 문서의 텍스트를 추출하고, 이를 의미적으로 응집력 있는 조각으로 분할한 뒤, 각 조각을 벡터로 변환하여 Chroma 벡터 저장소에 저장하게 됩니다. 
일반 유사도 기반 검색과 MMR 검색을 비교해보겠습니다. 
먼저 일반적인 유사도 기반 검색을 수행한 후, 최대 한계 관련성(Maximum Marginal Relevance, MMR) 검색을 사용하는 과정을 살펴봅니다.

In [80]:
# 1. 일반적인 유사도 기반 검색
query = '카카오뱅크의 환경목표와 세부추진내용을 알려줘?'
docs = db2.similarity_search(query)
print(len(docs))
print(docs[0].page_content)


4
1) 실사용량 집계가 가능한 오피스·데이터센터만 측정 대상에 포함
환경영향 최소화
카카오뱅크는 인터넷전문은행으로 
환경경영을 통해 친환경 가치를 실현하고 
있습니다. 온실가스 배출권 거래제의 
의무대상은 아니지만, 환경영향을 
최소화하고 리스크를 관리하기 위해 
자발적으로 온실가스 배출량을 산출하고 
제3자 검증을 거쳐 공개하고 있습니다. 
앞으로도 카카오뱅크는 환경 데이터 
집계와 탄소 배출량 관리를 통해 환경에 
미치는 부정적인 영향을 최소화할 수 있는 
방안을 모색해 나가겠습니다.


In [81]:
# 다음은 검색 결과 중 가장 유사도가 낮은(또는 마지막에 위치한) 문서의 내용을 출력합니다.
print(docs[-1].page_content)


재무적인 경로를 분석하여 향후 기후리스크에 대한 선제적이고 체계적인 대응을 위한 준비를 시작하였습니다.
카카오뱅크는 투자의사결정 시 환경, 사회, 지배구조 요소가 충분히 고려되도록 투자 가이드라인에 ESG 요소를 포함합니다. 2024년 3월까지 
카카오뱅크는 ESG 등급이 우수한 기업과 ESG 목적 채권 등으로 구성된 ESG 펀드에 약 700억 원을 투자하였고 추후 ESG 체크리스트를 개발하는 등 
ESG 투자 방식을 고도화할 예정입니다.
카카오뱅크는 친환경 건축물 인증제도(LEED)를 받은 판교테크원에 입주하여 용수사용량을 저감하고 고효율 LED 및 자동점등 센서를 사용하여 에너지를 
저감했습니다. 고객 대상으로는 교통카드 온라인 충전 서비스를 제공하여 종이영수증 발행을 감소시켰으며, 추후에는 고객의 친환경 활동에 대해 
우대금리를 제공하는 상품을 개발할 예정입니다. 
27~35, 80
기후변화 관련 위험과 기회가 사업, 전략 및 재무 계획에 미치는 영향
2℃ 이하 시나리오 등 기후변화 관련 시나리오를 고려한 조직 회복탄력성
위험관리
기후변화 관련 위험을 식별 및 평가하기 위한 프로세스
카카오뱅크는 중요 리스크에 기후리스크를 포함한 바 있으며, 이를 근거로 향후 기업 대출 강화 등 포트폴리오 다변화에 따른 기후리스크 평가 시스템을 
구축해 나갈 예정입니다. 연 1회 환경영향평가를 실시하여 이해관계자 요구사항을 반영한 주요 리스크와 기회를 식별 및 관리하고 있습니다. 도출된 이슈를 
해결하기 위한 목표와 세부 추진계획을 세워 환경영향을 최소화하고 있으며, 관련 지표들을 모니터링하고 있습니다. 
또한 2023년에는 환경경영시스템 매뉴얼 및 가이드라인을 제정하여 환경성과를 향상 시키고 환경목표를 달성하기 위해 노력하고 있습니다.
카카오뱅크는 의무적인 기후 또는 환경 규제(배출권 거래제 등)에 적용을 받지 않고, Scope 1&2로 배출되는 탄소배출량이 배출권 거래제에서 요구하는


In [82]:
# 2. MMR 검색
# 동일한 쿼리를 사용하여 MMR 검색을 수행합니다. 
# 여기서는 k=4와 fetch_k=10을 설정하여, 상위 10개의 유사한 문서 중에서 서로 다른 정보를 제공하는 4개의 문서를 선택합니다.
mmr_docs = db2.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)


4
1) 실사용량 집계가 가능한 오피스·데이터센터만 측정 대상에 포함
환경영향 최소화
카카오뱅크는 인터넷전문은행으로 
환경경영을 통해 친환경 가치를 실현하고 
있습니다. 온실가스 배출권 거래제의 
의무대상은 아니지만, 환경영향을 
최소화하고 리스크를 관리하기 위해 
자발적으로 온실가스 배출량을 산출하고 
제3자 검증을 거쳐 공개하고 있습니다. 
앞으로도 카카오뱅크는 환경 데이터 
집계와 탄소 배출량 관리를 통해 환경에 
미치는 부정적인 영향을 최소화할 수 있는 
방안을 모색해 나가겠습니다.


#### 백터스토어와 메타데이터를 추가

In [83]:
# 1. 메타데이터 포함하여 문서 준비
# 먼저, 각 문서에 대한 메타데이터를 포함하는 구조를 준비합니다. 
# Document 클래스를 사용하여 문서 내용과 메타데이터를 함께 관리할 수 있습니다.
from langchain_core.documents import Document

documents = [
    Document(
        page_content="LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.",
        metadata={
            "title": "LangChain 소개",
            "author": "AI 개발자",
            "url": "http://example.com/langchain-intro"
        }
    ),
    Document(
        page_content="벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데 특화된 데이터베이스 시스템입니다.",
        metadata={
            "title": "벡터 데이터베이스 개요",
            "author": "데이터 과학자",
            "url": "http://example.com/vector-db-overview"
        }
    )
    # 추가 문서들...
]


In [84]:
# 2. Chroma 백터 스토어에 문서와 메타데이터 저장
# Chroma 벡터 스토어에 문서를 저장할 때, from_documents 메서드를 사용하여 문서와 메타데이터를 함께 저장합니다.
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings()

# Chroma 벡터 스토어에 문서와 메타데이터 저장
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # 벡터 스토어를 디스크에 저장
)


In [85]:
# 3. 유사성 검색 수행 및 메타데이터 활용
query = "LangChain이란 무엇인가요?"
results = vectorstore.similarity_search(query, k=2)

for doc in results:
    print(f"내용: {doc.page_content}")
    print(f"제목: {doc.metadata['title']}")
    print(f"저자: {doc.metadata['author']}")
    print(f"URL: {doc.metadata['url']}")
    print("---")


내용: LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.
제목: LangChain 소개
저자: AI 개발자
URL: http://example.com/langchain-intro
---
내용: 벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데 특화된 데이터베이스 시스템입니다.
제목: 벡터 데이터베이스 개요
저자: 데이터 과학자
URL: http://example.com/vector-db-overview
---


### FAISS
FAISS(Facebook AI Similarity Search)는 Facebook AI Research에 의해 개발된 라이브러리로, 대규모 벡터 데이터셋에서 유사도 검색을 빠르고 효율적으로 수행할 수 있게 해줍니다. 
FAISS는 특히 벡터의 압축된 표현을 사용하여 메모리 사용량을 최소화하면서도 검색 속도를 극대화하는 특징이 있습니다

#### 유사도 기반 검색 (Similarity search)
FAISS 기반의 벡터 스토어를 생성하고 Huggingface에서 한국어 임베딩 모델을 다운로드 받아서 검색하는 과정을 살펴 보겠습니다. 
먼저 faiss-cpu와 sentence-transformers 패키지를 설치합니다. 
FAISS는 CPU만 사용하는 버전(faiss-cpu)과 GPU를 지원하는 버전(faiss-gpu)으로 나뉘는데, 여기서는 CPU 버전을 설치하는 방법으로 설명합니다. sentence-transformers는 임베딩 모델을 허깅페이스에서 다운로드 받기 위해서 설치합니다.

```
pip install faiss-cpu sentence-transformers
```

In [86]:
# 벡터스토어 db 인스턴스를 생성
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)
# HuggingFaceEmbeddings 클래스를 사용하여 사전 학습된 임베딩 모델(jhgan/ko-sbert-nli)을 로드하고, FAISS.from_documents 메서드를 사용하여 문서 객체를 임베딩 벡터로 변환하여 벡터 저장소에 저장합니다

vectorstore = FAISS.from_documents(documents,
                                   embedding = embeddings_model,
                                   distance_strategy = DistanceStrategy.COSINE
                                  )
vectorstore

# 단, 문서 객체를 임베딩 벡터로 변환하여 벡터 저장소에 저장할 때, 모델의 입력 길이 제한을 고려해야 합니다. 
# jhgan/ko-sbert-nli 모델의 경우 최대 시퀀스 길이는 128 토큰입니다. (일반적인 BERT 기반 모델은 최대 시퀀스 길이가 512 토큰입니다.) 
# 최대 시퀀스 길이를 초과하는 입력 문장은 잘리거나 패딩 처리됩니다.

## RAG - Retriever
Retrieval Augmented Generation (RAG)에서 검색도구(Retrievers)는 벡터 저장소에서 문서를 검색하는 핵심 구성 요소입니다. 
LangChain은 간단한 의미 검색부터 성능 향상을 위한 다양한 고급 검색 알고리즘을 지원합니다.

RAG 시스템의 검색 품질은 최종 답변의 정확도에 직접적인 영향을 미칩니다. 
LangChain은 다양한 검색 전략을 제공하여 사용 사례에 맞는 최적의 검색 방식을 선택할 수 있습니다.


|기법|핵심 원리|장점|적합한 상황|
|------|---|----|----|
|Vector Store Retriever|벡터 유사도 검색|간단, 빠름|기본 RAG 구현|
|Multi Query Retriever|다중 쿼리 생성|검색 범위 확장|모호한 질문 처리|
|Contextual Compression|관련 내용만 추출|노이즈 제거, 비용 절감|긴 문서에서 핵심 추출|
|Ensemble Retriever|BM25 + 벡터 결합|키워드와 의미 검색 통합|전문 용어가 중요한 도메인|
|RAG-Fusion|RRF로 결과 병합|다양성과 관련성 균형|복잡한 질문|
|Decomposition|질문 분해|복잡한 질문 처리|다단계 추론 필요|
|Step Back|추상화 질문 생성|배경 지식 활용|구체적 세부사항 질문|
|HyDE|가상 문서 임베딩|질문-문서 갭 해소|짧은 질문, 긴 문서|

- 검색 기법 선택 가이드
    기본 검색
    - Vector Store Retriever
      - 모든 RAG 시스템의 기본. 단순하고 빠른 검색이 필요할 때
    - Multi Query Retriever
      - 사용자 질문이 다양하게 해석될 수 있을 때
    - Contextual Compression
      - 검색된 문서가 길고 관련 없는 내용이 많을 때

    고급 검색
    - Ensemble Retriever
        -  키워드 매칭이 중요한 기술/법률/의료 도메인
    - RAG-Fusion
        - 검색 결과의 다양성과 정확도를 모두 높이고 싶을 때
    - Decomposition
        - "A와 B의 차이점은?" 같은 복합 질문 처리
    - Step Back
        - "2023년 삼성전자 반도체 매출은?" 같이 구체적 사실 질문
    - HyDE
        - 짧은 질문으로 긴 문서를 검색할 때 (질문-문서 임베딩 불일치 해소)